In [85]:
from qutip import *
from numpy import *
from scipy.linalg import expm

class spins:
    def __init__(self, N):
        si = qeye(2)
        sx = sigmax()
        sy = sigmay()
        sz = sigmaz()

        self.xv = []
        self.yv = []
        self.zv = []

        for n in range(N):
            op_list = [si] * N

            op_list[n] = sx
            self.xv.append(tensor(op_list))

            op_list[n] = sy
            self.yv.append(tensor(op_list))

            op_list[n] = sz
            self.zv.append(tensor(op_list))
    
    def x(self,n):
        return self.xv[n-1]/2.
    def y(self,n):
        return self.yv[n-1]/2.
    def z(self,n):
        return self.zv[n-1]/2.


s = spins(4)


H  = -8.952 * s.z(1)*s.z(2)

H += -4.979 * s.z(1)*s.z(3)
H += -4.979 * s.z(2)*s.z(3)

H += -5.663 * s.z(1)*s.z(4)
H += -5.663 * s.z(2)*s.z(4)

H += -2.430 * s.z(1)
H += -2.430 * s.z(2)
H += -4.979 * s.z(3)
H += -0.208 * s.z(4)

H += -19.09 * s.x(3)
H += -4.302 * s.x(4)

H += 15.10 * s.x(3)*s.x(4)

eta = 0.8239

psi4 = cos(eta)*basis(2,0) + sin(eta)*basis(2,1)

U = (-1j*H).expm()

G= tensor(toffoli(),qeye(2))


In [87]:
psi = rand_ket(N = 8, dims = [[2,2,2], [1,1,1]])
psitot = tensor(psi,psi4)

fidelity(toffoli()*psi,ptrace(U*psitot,[0,1,2]))


0.9996567631171528

In [102]:
dontCareStates = psi4
dontCareIdentity = [qeye(2)]


dCS = dontCareStates
dCI = tensor(dontCareIdentity)

G = toffoli()
dimG = G.shape[0]

from math import log

CareStateDim = int(log(dimG,2))

def getGate (G): #extract non zero elements of the gate and save them in s
    
    s = []

    rows = G.shape[0]
    colums = G.shape[1]
    
    for i in range(rows):
        for j in range(colums):
            if G[i][0][j] != 0 :
                s.append([i,j])
    return s

def bin(a):  #binary representation of a number a: useful to write the computational basis 
    s=''                                          #inside the Fidelity
    t={'0':'000','1':'001','2':'010','3':'011',
       '4':'100','5':'101','6':'110','7':'111'}
    for c in oct(a)[1:]:
            s+=t[c]
    return s

def getBasis (a) : #get the basis states according to the binary of a: 10 -> |10>
    
    if a == 0:
          B = [basis(2,0)]*(CareStateDim)
          return tensor(B)
    
    c = bin(a)
    if dimG == 8:
        return tensor(basis(2,int(c[0])) , basis(2,int(c[1])), basis(2,int(c[2])))
    if dimG == 4:
        return tensor(basis(2,int(c[1])) , basis(2,int(c[2])))

    
def Fidelity ():  #Fidelity function
    
    s = getGate(G)
    Fid = 1./(dimG + 1)
    Udag = U.dag()
    
  
    for x in s :
        for y in s:
            
            #definition of the basis kets and bras.             
            bra_i = getBasis(x[0]).dag()
            ket_j = getBasis(y[0])
            ket_k = getBasis(x[1])
            bra_l = getBasis(y[1]).dag()
            
            Epsilon = U*tensor(ket_k*bra_l, dCS*dCS.dag())*Udag
            Eps_ijkl = bra_i*(Epsilon.ptrace([0,1,2]))*ket_j
            
            Gstar_ik = G[x[0],x[1]].conjugate()
            G_jl = G[y[0],y[1]]
            
            
            fidStep = (1./(dimG*(dimG+1)))*Gstar_ik*Eps_ijkl*G_jl     
            Fid += fidStep[0][0][0]
            
    return abs(Fid)

Fidelity()

0.99926079142791313